In [137]:
import os
import json

In [138]:
with open("../data/enriched-extended-issues.json") as file:
    data = json.load(file)

In [139]:
len(data)

10489

In [140]:
data[:2]

[{'done': True,
  'id': '1-05c83321',
  'body': '',
  'file': '...',
  'role': 'DEV',
  'time': '2021-07-14T16:36:00Z',
  'email': '...',
  'lines': '1-1',
  'owner': 'AlvarezNunez',
  'author': '...',
  'issueNo': '2',
  'estimate': '60',
  'ticketNo': '1',
  'issueLink': 'https://github.com/AlvarezNunez/Jisland/issues/2',
  'title': ''},
 {'done': True,
  'comments': 0,
  'state': 'open',
  'closed_at': None,
  'created_at': '2021-07-14T12:53:56Z',
  'updated_at': '2021-07-14T12:53:56Z',
  'assignees': [],
  'locked': False,
  'labels': [],
  'milestone': None,
  'author_association': 'COLLABORATOR',
  'code': {'changes': 6, 'additions': 3, 'deletions': 3},
  'all_commits': [{'comment_count': 3,
    'raw_stats': {'total': 6, 'additions': 3, 'deletions': 3},
    'files': [{'additions': 3,
      'changes': 6,
      'deletions': 3,
      'status': 'modified',
      'filename': '2.java'}]}],
  'referenced': 1,
  'id': '1-c4c48bfd',
  'body': 'The puzzle `1-c4c48bfd` from #1 has to be res

In [141]:
issueLink = 'https://github.com/AlvarezNunez/demo_pdd/issues/2'
rootLink = '/'.join(issueLink.split(os.sep)[:-1])
rootLink

'https://github.com/AlvarezNunez/demo_pdd/issues'

In [142]:
new_data = []
for d in data:
    issueLink = d.get('issueLink', None)
    ticketNo = d.get('ticketNo', None)
    if issueLink and issueLink != 'unknown':
        rootLink = '/'.join(issueLink.split(os.sep)[:-1])
        d['parentIssue'] = rootLink + '/' + ticketNo
        parent = [i for idx, i in enumerate(data) if i.get('issueLink', None) == d['parentIssue']]
        if len(parent) > 0:
            parent = parent[0]
            d['parentIssueData'] = { 'id': parent.get('id', 'None'), 'state': parent.get('state', None), 'issueLink': parent.get('issueLink', None) }
        else:
            d['parentIssueData'] = None
    else:
        d['issueLink'] = None
        d['parentIssue'] = None
        d['parentIssueData'] = None
    new_data.append(d)

In [143]:
data_set = dict();

In [144]:
new_data[:2]

[{'done': True,
  'id': '1-05c83321',
  'body': '',
  'file': '...',
  'role': 'DEV',
  'time': '2021-07-14T16:36:00Z',
  'email': '...',
  'lines': '1-1',
  'owner': 'AlvarezNunez',
  'author': '...',
  'issueNo': '2',
  'estimate': '60',
  'ticketNo': '1',
  'issueLink': 'https://github.com/AlvarezNunez/Jisland/issues/2',
  'title': '',
  'parentIssue': 'https://github.com/AlvarezNunez/Jisland/issues/1',
  'parentIssueData': None},
 {'done': True,
  'comments': 0,
  'state': 'open',
  'closed_at': None,
  'created_at': '2021-07-14T12:53:56Z',
  'updated_at': '2021-07-14T12:53:56Z',
  'assignees': [],
  'locked': False,
  'labels': [],
  'milestone': None,
  'author_association': 'COLLABORATOR',
  'code': {'changes': 6, 'additions': 3, 'deletions': 3},
  'all_commits': [{'comment_count': 3,
    'raw_stats': {'total': 6, 'additions': 3, 'deletions': 3},
    'files': [{'additions': 3,
      'changes': 6,
      'deletions': 3,
      'status': 'modified',
      'filename': '2.java'}]}],
 

In [145]:
for d in new_data:
    parentIssue = d.get('parentIssue', None)
    group = data_set.get(parentIssue, [])
    current = { 'id': d.get('id', 'None'), 'state': d.get('state', None), 'issueLink': d.get('issueLink', None) }
    group.append(current)
    data_set[parentIssue] = group

In [146]:
del data_set[None]

In [147]:
for parent, children in data_set.items():
    result = [(idx, i) for idx, i in enumerate(new_data) if i['issueLink'] == parent]
    if len(result) > 0:
        idx, issue = result[0]
        issue['childrenIssues'] = children
        new_data[idx] = issue

In [148]:
c = [x for x in new_data if len(x.get('childrenIssues', [])) > 0]

In [152]:
final_data = []
for d in new_data:
    if d['parentIssueData'] is None:
        d['parentIssue'] = { 'id': None, 'state': None, 'issueLink': d['parentIssue'] }
    else:
        d['parentIssue'] = d['parentIssueData']
    del d['parentIssueData']
    final_data.append(d)

In [155]:
with open("../data/enriched-extended-issues-with-relationship.json", 'w') as file:
    json.dump(final_data, file, indent=2)

